# 2D Matrix Multiplication using C++



In [ ]:
%%writefile ElementWiseC.c
#include <stdlib.h>
#include <time.h>
#include <stdio.h>
#include "parameters.c"  // contains # of elements

void ElemW(float* x, float* y, float* z, int N) {
    for (int i = 0; i < N; i++)  // i = column
    {
        for (int j = 0; j < N; j++)  // j = rows
        {
            int offset = i * N + j;
            *(z + offset) = *(y + offset) * *(x + offset);
        }
    }
}

void verifyMatrixMultiplication(float* x, float* y, float* z, int N) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            int offset = i * N + j;
            float expected = y[offset] * x[offset];
            if (z[offset] != expected) {
                printf("Verification failed at index (%d, %d).\n", i, j);
                printf("Expected: %f, Actual: %f\n", expected, z[offset]);
                return;
            }
        }
    }
    printf("Verification successful. Matrix multiplication is correct.\n");
}

int main() {
    const int ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    float* x, * y, * z;

    z = (float*)malloc(ARRAY_BYTES);
    x = (float*)malloc(ARRAY_BYTES);
    y = (float*)malloc(ARRAY_BYTES);

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            int offset = i * N + j;
            *(x + offset) = 2.0;
        }
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            int offset = i * N + j;
            *(y + offset) = j;
        }
    }

    clock_t start, end;
    double time_taken, sum = 0, ave = 0;
    for (int i = 0; i < loops; i++) {
        start = clock();  // record time
        ElemW(x, y, z, N);
        end = clock();
        time_taken = ((double)(end - start)) * 1e6 / CLOCKS_PER_SEC;
        sum += time_taken;  // total execution time
    }

    ave = sum / loops;  // average execution time

    printf("ARRAYSIZE of %d x %d Average time: %f microseconds\n", N, N, ave);
    printf("Element Wise Product Test for matrix of 2's multiplied matrix of 1 to N...\n");

    verifyMatrixMultiplication(x, y, z, N);

    free(x);
    free(y);
    free(z);

    return 0;
}


Writing ElementWiseC.c


## N = 1024

In [ ]:
%%writefile parameters.c
const int N = 1024; //number of elements
const int ARRAY_SIZE = N*N;
const int loops = 30;

Overwriting parameters.c


In [ ]:
%%shell
g++ ElementWiseC.c -o ElementWiseC
./ElementWiseC

ARRAYSIZE of 1024 x 1024 Average time: 4313.733333 microseconds
Element Wise Product Test for matrix of 2's multiplied matrix of 1 to N...
Verification successful. Matrix multiplication is correct.


## N = 2048



In [ ]:
%%writefile parameters.c
const int N = 2048; //number of elements
const int ARRAY_SIZE = N*N;
const int loops = 30;

Overwriting parameters.c


In [ ]:
%%shell
g++ ElementWiseC.c -o ElementWiseC
./ElementWiseC

ARRAYSIZE of 2048 x 2048 Average time: 15061.766667 microseconds
Element Wise Product Test for matrix of 2's multiplied matrix of 1 to N...
Verification successful. Matrix multiplication is correct.


## N = 4096

In [ ]:
%%writefile parameters.c
const int N = 4096; //number of elements
const int ARRAY_SIZE = N*N;
const int loops = 31;

Overwriting parameters.c


In [ ]:
%%shell
g++ ElementWiseC.c -o ElementWiseC
./ElementWiseC

ARRAYSIZE of 4096 x 4096 Average time: 61957.225806 microseconds
Element Wise Product Test for matrix of 2's multiplied matrix of 1 to N...
Verification successful. Matrix multiplication is correct.


# 1D Matrix Multiplication using CUDA

In [ ]:
%%writefile Cuda1D.cu 
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "parameters.c"

__global__ 
void matrixMul(float *x, float *y, float *z, int N)
{ 
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for(int i = tid; i < N; i += stride)
    {
        float sum = 0.0f;
        for (int j = 0; j < N; j++) {
            sum += x[i * N + j] * y[j];
        }
        z[i] = sum;
    }
}

void verifyMatrixMultiplication(float *x, float *y, float *z, int N) {
    for (int i = 0; i < N; i++) {
        float expected = 0.0f;
        for (int j = 0; j < N; j++) {
            expected += x[i * N + j] * y[j];
        }
        if (z[i] != expected) {
            printf("Verification failed at index %d.\n", i);
            printf("Expected: %f, Actual: %f\n", expected, z[i]);
            return;
        }
    }
    printf("Verification successful. Matrix multiplication is correct.\n");
}

int main()
{
    const int ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    float *x, *y, *z;
    int blocks = (ARRAY_SIZE + bsize - 1) / bsize;  
    cudaMallocManaged(&x, ARRAY_BYTES);
    cudaMallocManaged(&y, ARRAY_BYTES);
    cudaMallocManaged(&z, ARRAY_BYTES);

    for (int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            int offset = i * N + j;
            x[offset] = 2.0f;
        }
        y[i] = 1.0f;
    }

    cudaMemAdvise(x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, 0);
    cudaMemAdvise(y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, 0);

    cudaMemPrefetchAsync(x, ARRAY_BYTES, cudaCpuDeviceId);
    cudaMemPrefetchAsync(y, ARRAY_BYTES, cudaCpuDeviceId);

    for (int i = 0; i < loops; i++)
    {
        matrixMul<<<blocks, threadsPerBlock>>>(x, y, z, N);
        cudaDeviceSynchronize();
    }

    verifyMatrixMultiplication(x, y, z, N);

    cudaFree(x);
    cudaFree(y);
    cudaFree(z);

    return 0;
}


Writing Cuda1D.cu


## Vector Size = 2^20 , Threads per block = 256

In [ ]:
%%writefile parameters.c
dim3 threadsPerBlock(256,1,1);
const int bsize = 256;
const int N = 1024; //number of elements
const int ARRAY_SIZE = N*N;
const int loops = 30;


Overwriting parameters.c


In [ ]:
%%shell
nvcc Cuda1D.cu -o Cuda1D
nvprof ./Cuda1D

==7033== NVPROF is profiling process 7033, command: ./Cuda1D
Verification successful. Matrix multiplication is correct.
==7033== Profiling application: ./Cuda1D
==7033== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  30.953ms        30  1.0318ms  977.48us  2.4564ms  matrixMul(float*, float*, float*, int)
      API calls:   84.79%  187.45ms         3  62.484ms  10.291us  187.40ms  cudaMallocManaged
                   14.06%  31.083ms        30  1.0361ms  964.89us  2.4639ms  cudaDeviceSynchronize
                    0.49%  1.0884ms         2  544.18us  21.283us  1.0671ms  cudaMemPrefetchAsync
                    0.34%  741.97us         3  247.32us  64.289us  392.96us  cudaFree
                    0.13%  291.38us        30  9.7120us  4.4420us  32.789us  cudaLaunchKernel
                    0.10%  226.34us         2  113.17us  12.158us  214.18us  cudaMemAdvise
                    0.07%  147.88us       101  1.46

## Vector Size = 2^22 , Threads per block = 512

In [ ]:
%%writefile parameters.c
dim3 threadsPerBlock(512,1,1);
const int bsize = 512;
const int N = 2048; //number of elements
const int ARRAY_SIZE = N*N;
const int loops = 30;


Overwriting parameters.c


In [ ]:
%%shell
nvcc Cuda1D.cu -o Cuda1D
nvprof ./Cuda1D

==7341== NVPROF is profiling process 7341, command: ./Cuda1D
Verification successful. Matrix multiplication is correct.
==7341== Profiling application: ./Cuda1D
==7341== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  127.37ms        30  4.2455ms  3.9373ms  12.708ms  matrixMul(float*, float*, float*, int)
      API calls:   51.46%  143.29ms         3  47.765ms  20.462us  143.24ms  cudaMallocManaged
                   45.81%  127.54ms        30  4.2515ms  3.9435ms  12.716ms  cudaDeviceSynchronize
                    1.54%  4.2769ms         2  2.1385ms  21.191us  4.2557ms  cudaMemPrefetchAsync
                    0.69%  1.9160ms         3  638.67us  61.682us  1.1991ms  cudaFree
                    0.30%  840.41us         2  420.20us  15.717us  824.69us  cudaMemAdvise
                    0.14%  396.92us        30  13.230us  5.9480us  35.738us  cudaLaunchKernel
                    0.05%  131.61us       101  1.30

## Vector Size = 2^24 , Threads per block = 1024

In [ ]:
%%writefile parameters.c
dim3 threadsPerBlock(1024,1,1);
const int bsize = 1024;
const int N = 4096; //number of elements
const int ARRAY_SIZE = N*N;
const int loops = 30;

In [ ]:
%%shell
nvcc Cuda1D.cu -o Cuda1D
nvprof ./Cuda1D

# 2D Matrix Multiplication using CUDA

In [ ]:
%%writefile ElementWiseCU.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "parameters.c"

__global__ 
void ElementWise(float *x, float *y, float *z, int N)
{ 
    int ix, iy, stridex, stridey, offset;
    ix = blockIdx.x * blockDim.x + threadIdx.x;
    iy = blockIdx.y * blockDim.y + threadIdx.y;
    stridex = blockDim.x * gridDim.x;
    stridey = blockDim.y * gridDim.y;
    
    for(int i = ix; i < N; i += stridex)
    {
        for (int j = iy; j < N; j += stridey)
        {
            offset = i * N + j;
            *(z + offset) = *(x + offset) * *(y + offset);
        }
    }
}

void verifyMatrixMultiplication(float *x, float *y, float *z, int N) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            float expected = x[i * N + j] * y[i * N + j];
            if (z[i * N + j] !=A expected) {
                printf("Verification failed at index (%d, %d).\n", i, j);
                printf("Expected: %f, Actual: %f\n", expected, z[i * N + j]);
                return;
            }
        }
    }
    printf("Verification successful. Matrix multiplication is correct.\n");
}

int main()
{
    const int ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    float *x, *y, *z;
    int blocks = (ARRAY_SIZE + bsize - 1) / bsize;  
    cudaMallocManaged(&x, ARRAY_BYTES);
    cudaMallocManaged(&y, ARRAY_BYTES);
    cudaMallocManaged(&z, ARRAY_BYTES);
    
    for(int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            int offset = i * N + j;
            *(x + offset) = float(2);
        }
    }

    for(int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            int offset = i * N + j;
            *(y + offset) = float(j);
        }
    }

    cudaMemAdvise(x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, 0);
    cudaMemAdvise(y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, 0);

    cudaMemPrefetchAsync(x, ARRAY_BYTES, cudaCpuDeviceId);
    cudaMemPrefetchAsync(y, ARRAY_BYTES, cudaCpuDeviceId);

    for (int i = 0; i < loops; i++)
    {
        ElementWise<<<blocks, threadsPerBlock>>>(x, y, z, N); 
        cudaDeviceSynchronize();   
    }

    verifyMatrixMultiplication(x, y, z, N);

    cudaFree(x);
    cudaFree(y);
    cudaFree(z);

    return 0;
}


Writing ElementWiseCU.cu


## Vector Size = 1024x1024 , Threads per block = 8x8

In [ ]:
%%writefile parameters.c
dim3 threadsPerBlock(8,8,1);
const int bsize = 64;
const int N = 1024; //number of elements
const int ARRAY_SIZE = N*N;
const int loops = 30;


Writing parameters.c


In [ ]:
%%shell
nvcc ElementWiseCU.cu -o ElementWiseCU
nvprof ./ElementWiseCU

==6395== NVPROF is profiling process 6395, command: ./ElementWiseCU
Verification successful. Matrix multiplication is correct.
==6395== Profiling application: ./ElementWiseCU
==6395== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  6.9524ms        30  231.75us  131.87us  3.0431ms  ElementWise(float*, float*, float*, int)
      API calls:   96.74%  291.60ms         3  97.200ms  10.107us  291.56ms  cudaMallocManaged
                    2.35%  7.0879ms        30  236.26us  136.12us  3.0497ms  cudaDeviceSynchronize
                    0.36%  1.0879ms         1  1.0879ms  1.0879ms  1.0879ms  cuDeviceGetPCIBusId
                    0.29%  873.97us         3  291.32us  276.00us  314.22us  cudaFree
                    0.14%  412.39us         2  206.20us  192.48us  219.91us  cudaMemAdvise
                    0.07%  199.02us        30  6.6330us  3.3520us  55.880us  cudaLaunchKernel
                    0.04%  123.08us 

## Vector Size = 2048x2048 , Threads per block = 16x16

In [ ]:
%%writefile parameters.c
dim3 threadsPerBlock(16,16,1);
const int bsize = 256;
const int N = 2048; //number of elements
const int ARRAY_SIZE = N*N;
const int loops = 30;

Overwriting parameters.c


In [ ]:
%%shell
nvcc ElementWiseCU.cu -o ElementWiseCU
nvprof ./ElementWiseCU

==6931== NVPROF is profiling process 6931, command: ./ElementWiseCU
Verification successful. Matrix multiplication is correct.
==6931== Profiling application: ./ElementWiseCU
==6931== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  35.835ms        30  1.1945ms  694.89us  15.632ms  ElementWise(float*, float*, float*, int)
      API calls:   78.53%  153.92ms         3  51.306ms  20.386us  153.86ms  cudaMallocManaged
                   18.35%  35.970ms        30  1.1990ms  697.64us  15.641ms  cudaDeviceSynchronize
                    2.06%  4.0433ms         3  1.3478ms  1.2988ms  1.4013ms  cudaFree
                    0.83%  1.6183ms         2  809.14us  784.52us  833.76us  cudaMemAdvise
                    0.13%  247.37us        30  8.2450us  4.1990us  42.370us  cudaLaunchKernel
                    0.07%  129.41us       101  1.2810us     150ns  54.365us  cuDeviceGetAttribute
                    0.02%  42.633us

## Vector Size = 4096x4096 , Threads per block = 32x32

In [ ]:
%%writefile parameters.c
dim3 threadsPerBlock(32,32,1);
const int bsize = 1024;
const int N = 4096; //number of elements
const int ARRAY_SIZE = N*N;
const int loops = 30;

Overwriting parameters.c


In [ ]:
%%shell
nvcc ElementWiseCU.cu -o ElementWiseCU
nvprof ./ElementWiseCU

==7141== NVPROF is profiling process 7141, command: ./ElementWiseCU
Verification successful. Matrix multiplication is correct.
==7141== Profiling application: ./ElementWiseCU
==7141== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  221.00ms        30  7.3668ms  2.3571ms  50.416ms  ElementWise(float*, float*, float*, int)
      API calls:   56.18%  221.83ms        30  7.3944ms  2.3598ms  50.431ms  cudaDeviceSynchronize
                   37.03%  146.24ms         3  48.747ms  13.860us  146.18ms  cudaMallocManaged
                    4.76%  18.814ms         3  6.2713ms  5.0322ms  7.2423ms  cudaFree
                    1.85%  7.3163ms         2  3.6582ms  3.1626ms  4.1538ms  cudaMemAdvise
                    0.12%  455.46us        30  15.182us  5.9100us  41.374us  cudaLaunchKernel
                    0.03%  120.42us       101  1.1920us     127ns  52.160us  cuDeviceGetAttribute
                    0.02%  78.453us